In [27]:
import pandas as pd
import requests
import bs4
from bs4 import BeautifulSoup
import time
import numpy as np
import os

In [2]:
def find_url_table_4(soup,base_url):
    Flag = 'None'
    soup = soup('a')
    for a in soup:
        if a.text.strip().__contains__('Table 4'):
            link = a['href']
            Flag = 'Have'

    if (Flag == 'Have'):
        url = base_url + link
        return url
    else:
        return 'Last page'

In [3]:
def create_country_list(soup):
    country_list = list()
    n = soup('strong')
    for x in n:
        if not x.text.strip().__contains__('Table 4'):
            country_list.append((x.text.strip()))
    return country_list

In [4]:
def create_drink_lists(soup):
    beer_list = list()
    wine_list = list()
    spirits_list = list()
    beverages_list = list()

    beer_spirits = soup('td', attrs={'class': 'datagray'})
    wine_beverages = soup('td', attrs={'class': 'data'})

    for x in range(1, beer_spirits.__len__(), 3):
        beer_list.append(beer_spirits[x-1].text.strip())
        spirits_list.append(beer_spirits[x].text.strip())

    for x in range(1, wine_beverages.__len__(), 2):
        wine_list.append(wine_beverages[x - 1].text.strip())
        beverages_list.append(wine_beverages[x].text.strip())

    return beer_list, wine_list, spirits_list, beverages_list

In [19]:
def saving_data_to_df(beer_list, wine_list, spirits_list, beverages_list, country_list):
    rows = country_list.__len__() * 4
    cols = 45  # all years + 1 for type + 1 country.
    matrix = np.zeros((rows, cols))
    r = 0
    for x in range(3, rows, 4):
        matrix[x - 3][1] = 1
        matrix[x - 2][1] = 2
        matrix[x - 1][1] = 3
        matrix[x][1] = 4
        for c in range(2, cols):
            matrix[x - 3][c] = beer_list[r]
            matrix[x - 2][c] = wine_list[r]
            matrix[x - 1][c] = spirits_list[r]
            matrix[x][c] = beverages_list[r]
            r += 1

    df = pd.DataFrame(matrix)
    country_list_x4 = list()
    for country in country_list:
        country_list_x4 += 4 * [country]

    df[1] = df[1].astype(int)
    df[0] = df[0].astype(object)
    df[0] = country_list_x4
    return df

In [20]:
def load_soup_object(url):
    source_code = requests.get(url)
    plain_text = source_code.text
    #soup = BeautifulSoup(plain_text)
    soup = BeautifulSoup(plain_text,features="lxml") # added...
    return soup

In [21]:
def save_csv(dataframe, csv_name, folder_name):
    csv_path = folder_name + os.sep + csv_name
    dataframe.to_csv(csv_path)

In [22]:
base_url='https://pubs.niaaa.nih.gov/publications/'
url = base_url + 'surveillance.htm'
soup = load_soup_object(url)

country_list = list()
#drink_list = list()

In [24]:
soup = soup('a', attrs={'target': '_blank'})
for a in soup:
    if a.text.strip().__contains__('Surveillance Report #117'):
        link = a['href']
url = base_url + link
soup = load_soup_object(url)
base_url = base_url + 'surveillance117/'

In [25]:
url = find_url_table_4(soup, base_url)
soup = load_soup_object(url)
country_list = create_country_list(soup)
beer_list, wine_list, spirits_list, beverages_list = create_drink_lists(soup)
df1 = saving_data_to_df(beer_list, wine_list, spirits_list, beverages_list, country_list)
url = find_url_table_4(soup, base_url)
#soup = load_soup_object(url)
while (url != 'Last page'):
#soup = load_soup_object(url)
    soup = load_soup_object(url)
    country_list = create_country_list(soup)
    beer_list, wine_list, spirits_list, beverages_list = create_drink_lists(soup)
    df2 = saving_data_to_df(beer_list, wine_list, spirits_list, beverages_list, country_list)
    df1 = [df1, df2]
    df1 = pd.concat(df1)
    url = find_url_table_4(soup, base_url)

In [29]:
df3=df1.rename(columns={0: 'Country', 1: 'Drink type'},inplace = False)
for i in range(2, 45):
    df3= df3.rename(columns = {i: (2019-i+2)}, inplace = False)

index_list = list()
for i in range(0,224):
    index_list.append(i)

df3.index = index_list
save_csv(df3, 'niaaa_nih.csv', 'niaaa.nih')

In [ ]:
df